In [264]:
import pygsti
from pygsti.circuits import Circuit
import random
from pygsti.modelpacks import smq1Q_XYZI as std
# Note that numpy 1.23 will break pygsti
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from forest.benchmarking.operator_tools import vec, unvec

In [265]:
model = std.target_model('CPTP')

In [330]:
from forest.benchmarking.utils import *
from forest.benchmarking.operator_tools import *

class ErrorGen():
    """
    Implements the small Markovian taxonomy
    
    Parameters:
        -features: pauli transfer representations for different error processes
        -rates: sympy vector of rates
    """
    
    def __init__(self, num_qubits, label=('G', 0), parameterization='HSCA'):
        """
        setup the feature dictionary with keys that are sympy variables
        
        right now, I'm doing this by hand for a 1 qubit model, 
        TODO: implement logic to generalize to n-qubits
        """
        self.num_qubits = num_qubits
        self.basis = n_qubit_pauli_basis(num_qubits)
        self.feature_map = dict()
        gate_label = label[0]
        qbit_index = label[1]
        if 'H' in parameterization:
            self.hparams = sp.symbols(
                f'H^{{X{qbit_index}}}_{{{gate_label}}} ' +
                f'H^{{Y{qbit_index}}}_{{{gate_label}}} ' +  
                f'H^{{Z{qbit_index}}}_{{{gate_label}}}')
            self.feature_map[self.hparams[0]] = self.herr('X')
            self.feature_map[self.hparams[1]] = self.herr('Y')
            self.feature_map[self.hparams[2]] = self.herr('Z')
        else:
            self.hparams = []
            
        if 'S' in parameterization:
            self.sparams = sp.symbols(
                f'S^{{X{qbit_index}}}_{{{gate_label}}} ' +
                f'S^{{Y{qbit_index}}}_{{{gate_label}}} ' +  
                f'S^{{Z{qbit_index}}}_{{{gate_label}}}')
            self.feature_map[self.sparams[0]] = self.serr('X')
            self.feature_map[self.sparams[1]] = self.serr('Y')
            self.feature_map[self.sparams[2]] = self.serr('Z')
        else:
            self.sparams = []
            
        if 'C' in parameterization:
            self.cparams = sp.symbols(
                f'C^{{XY{qbit_index}}}_{{{gate_label}}} ' +
                f'C^{{YZ{qbit_index}}}_{{{gate_label}}} ' +  
                f'C^{{ZX{qbit_index}}}_{{{gate_label}}}')
            self.feature_map[self.cparams[0]] = self.cerr('X', 'Y')
            self.feature_map[self.cparams[1]] = self.cerr('Y', 'Z')
            self.feature_map[self.cparams[2]] = self.cerr('Z', 'X')
        else:
            self.cparams = []
            
        if 'A' in parameterization:
            self.aparams = sp.symbols(
                f'A^{{XY{qbit_index}}}_{{{gate_label}}} ' +
                f'A^{{YZ{qbit_index}}}_{{{gate_label}}} ' +  
                f'A^{{ZX{qbit_index}}}_{{{gate_label}}}')
            self.feature_map[self.aparams[0]] = self.aerr('X', 'Y')
            self.feature_map[self.aparams[1]] = self.aerr('Y', 'Z')
            self.feature_map[self.aparams[2]] = self.aerr('Z', 'X')
        else:
            self.aparams = []
            
    @property
    def vec(self):
        return self.hparams+self.sparams+self.cparams+self.aparams
    
    @property
    def mat(self):
        mat = np.zeros((4**self.num_qubits, 4**self.num_qubits), dtype='O')
        for x in self.vec:
            mat += x*self.feature_map[x]
        return sp.Matrix(mat)
    
    @property
    def features(self):
        return [f for f in self.feature_map.values()]
    
    def herr(self, pauli_str):
        """
        hamiltonian error labelled by pauli
        
        Equation (10) in the paper
        """
        pindex = self.basis.labels.index(pauli_str)
        chi_mat = np.zeros((4**len(pauli_str), 4**len(pauli_str)), dtype=complex)
        chi_mat[0, pindex] = 1j
        chi_mat[pindex, 0] = -1j
        return chi2pauli_liouville(chi_mat)
    
    def serr(self, pauli_str):
        """
        stochatic error labelled by pauli
        
        Equation (14) in the paper
        """
        pindex = self.basis.labels.index(pauli_str)
        chi_mat = np.zeros((4**len(pauli_str), 4**len(pauli_str)), dtype=complex)
        chi_mat[0, 0] = -1
        chi_mat[pindex, pindex] = 1
        return chi2pauli_liouville(chi_mat)
    
    def cerr(self, pauli_str1, pauli_str2):
        """
        correlated stochatic error labelled by pair of paulis
        
        Equation (15) in the paper
        """
        pindex1 = self.basis.labels.index(pauli_str1)
        pindex2 = self.basis.labels.index(pauli_str2)
        
        # we also need product terms and their phases
        # this could probably be made more elegant
        pterm1 = str_to_pauli_term(pauli_str1)*str_to_pauli_term(pauli_str2)
        pterm2 = str_to_pauli_term(pauli_str2)*str_to_pauli_term(pauli_str1)
        
        phase1 = pterm1.coefficient
        phase2 = pterm2.coefficient
        pid1 = pterm1.id()
        pid2 = pterm1.id()
        # need to fix the id convention that pyquil uses
        # discard the even indices
        pid1 = [ele for idx, ele in enumerate(pid1) if idx % 2 == 0]
        pid2 = [ele for idx, ele in enumerate(pid2) if idx % 2 == 0]
        prod_index1 = self.basis.labels.index(''.join(pid1))
        prod_index2 = self.basis.labels.index(''.join(pid2))

        chi_mat = np.zeros((4**self.num_qubits, 4**self.num_qubits), dtype=complex)
        chi_mat[pindex1, pindex2] += 1
        chi_mat[pindex2, pindex1] += 1
        # include the terms 
        # PQrII + QPrII + IIrPQ + IIrQP
        chi_mat[prod_index1, 0] += -0.5*phase1
        chi_mat[prod_index2, 0] += -0.5*phase2
        chi_mat[0, prod_index1] += -0.5*phase1
        chi_mat[0, prod_index2] += -0.5*phase2
        return chi2pauli_liouville(chi_mat)
    
    def aerr(self, pauli_str1, pauli_str2):
        """
        active error labelled by pair of paulis
        
        Equation (16) in the paper
        """
        pindex1 = self.basis.labels.index(pauli_str1)
        pindex2 = self.basis.labels.index(pauli_str2)
        
        # we also need product terms and their phases
        # this could probably be made more elegant
        pterm1 = str_to_pauli_term(pauli_str1)*str_to_pauli_term(pauli_str2)
        pterm2 = str_to_pauli_term(pauli_str2)*str_to_pauli_term(pauli_str1)
        
        phase1 = pterm1.coefficient
        phase2 = pterm2.coefficient
        pid1 = pterm1.id()
        pid2 = pterm1.id()
        # need to fix the id convention that pyquil uses
        # discard the even indices
        pid1 = [ele for idx, ele in enumerate(pid1) if idx % 2 == 0]
        pid2 = [ele for idx, ele in enumerate(pid2) if idx % 2 == 0]
        prod_index1 = self.basis.labels.index(''.join(pid1))
        prod_index2 = self.basis.labels.index(''.join(pid2))

        chi_mat = np.zeros((4**self.num_qubits, 4**self.num_qubits), dtype=complex)
        chi_mat[pindex1, pindex2] += 1j
        chi_mat[pindex2, pindex1] += -1j
        # include the terms 
        # PQrII - QPrII + IIrPQ - IIrQP
        chi_mat[prod_index1, 0] += 0.5j*phase1
        chi_mat[prod_index2, 0] += -0.5j*phase2
        chi_mat[0, prod_index1] += 0.5j*phase1
        chi_mat[0, prod_index2] += -0.5j*phase2
        return chi2pauli_liouville(chi_mat)

In [331]:
egen = ErrorGen(1, c[1])
egen.hparams[1]
egen.feature_map[egen.hparams[1]]

array([[ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  2.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j, -2.+0.j,  0.+0.j,  0.+0.j]])

In [332]:
egen.vec

(H^{X0}_{Gxpi2},
 H^{Y0}_{Gxpi2},
 H^{Z0}_{Gxpi2},
 S^{X0}_{Gxpi2},
 S^{Y0}_{Gxpi2},
 S^{Z0}_{Gxpi2},
 C^{XY0}_{Gxpi2},
 C^{YZ0}_{Gxpi2},
 C^{ZX0}_{Gxpi2},
 A^{XY0}_{Gxpi2},
 A^{YZ0}_{Gxpi2},
 A^{ZX0}_{Gxpi2})

In [333]:
egen.mat

Matrix([
[                   0,                                        0,                                        0,                                        0],
[-4.0*A^{YZ0}_{Gxpi2}, -2.0*S^{Y0}_{Gxpi2} - 2.0*S^{Z0}_{Gxpi2}, 2.0*C^{XY0}_{Gxpi2} - 2.0*H^{Z0}_{Gxpi2}, 2.0*C^{ZX0}_{Gxpi2} + 2.0*H^{Y0}_{Gxpi2}],
[-4.0*A^{ZX0}_{Gxpi2}, 2.0*C^{XY0}_{Gxpi2} + 2.0*H^{Z0}_{Gxpi2}, -2.0*S^{X0}_{Gxpi2} - 2.0*S^{Z0}_{Gxpi2}, 2.0*C^{YZ0}_{Gxpi2} - 2.0*H^{X0}_{Gxpi2}],
[-4.0*A^{XY0}_{Gxpi2}, 2.0*C^{ZX0}_{Gxpi2} - 2.0*H^{Y0}_{Gxpi2}, 2.0*C^{YZ0}_{Gxpi2} + 2.0*H^{X0}_{Gxpi2}, -2.0*S^{X0}_{Gxpi2} - 2.0*S^{Y0}_{Gxpi2}]])

In [335]:
"""print out all the elementary generators for 1 qubit egen"""
for x in egen.vec:
    print(x, '\n', egen.feature_map[x])

H^{X0}_{Gxpi2} 
 [[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -2.+0.j]
 [ 0.+0.j  0.+0.j  2.+0.j  0.+0.j]]
H^{Y0}_{Gxpi2} 
 [[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  2.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -2.+0.j  0.+0.j  0.+0.j]]
H^{Z0}_{Gxpi2} 
 [[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -2.+0.j  0.+0.j]
 [ 0.+0.j  2.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]]
S^{X0}_{Gxpi2} 
 [[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -2.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -2.+0.j]]
S^{Y0}_{Gxpi2} 
 [[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -2.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -2.+0.j]]
S^{Z0}_{Gxpi2} 
 [[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -2.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -2.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]]
C^{XY0}_{Gxpi2} 
 [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0

In [376]:
class GateSet():
    """
    Holds the information for
        -all native gates, states, and measurements
        -all error generator features (in a feature dict)
        -map between gate labels and parameter index list (index set)
        -map between parameter indices and error features (feature dict) 
        
    TODO: generalize to n-qubits
    """
    def __init__(self, model_pack, parameterization='HSCA'):
        # A list of error rates, each one corresponding to one feature in the above list
        self.rates = []
        self.feature_map = dict() # maps variables to their associated feature
        self.egen_from_label = dict() # maps gate labels to error generators
        self.target_model = model_pack.target_model('CPTP')
        self.gate_labels = model_pack.gates
        
        # Errors generators for all gates in the model
        for gate in model_pack.gates:
            if len(gate) <= 1:
                egen = ErrorGen(1, (gate, ''))
            else:
                egen = ErrorGen(1, gate)
            self.egen_from_label[gate] = egen
            self.rates += egen.vec
            self.feature_map.update(egen.feature_map)
        # Errors for all measurements in the model
        for povm in model.povms.keys():
            egen = ErrorGen(1, (povm, 'M'))
            self.egen_from_label[povm] = egen
            self.rates += egen.vec
            self.feature_map.update(egen.feature_map)
        # Errors for all state preparations in the model
        for prep in model.preps.keys():
            egen = ErrorGen(1, (prep, 'P'))
            self.egen_from_label[prep] = egen
            self.rates += egen.vec
            self.feature_map.update(egen.feature_map)
            
    @property
    def vec(self):
        return self.rates
    
    def features_of(self, op_label):
        """Returns the list of error features associated with the given gate/POVM/state-prep"""
        return self.egen_from_label[op_label].features
    
    def rates_of(self, op_label):
        """Returns the list of error rates associated with the given gate/POVM/state-prep"""
        return self.egen_from_label[op_label].vec
    
    def error_generator_of(self, op_label):
        """Returns the error generator matrix associated with the given gate/POVM/state-prep"""
        return self.egen_from_label[op_label].mat
    
    def target_op(self, op_label):
        """
        returns the target operation of the given operation
        """
        return self.target_model[op_label].to_dense()

    def native_meas(self):
        return [m.to_dense() for m in self.target_model['Mdefault'].values()]

    def rho0(self):
        return self.target_model['rho0'].to_dense()


In [377]:
gs.vec

[H^{X}_{()},
 H^{Y}_{()},
 H^{Z}_{()},
 S^{X}_{()},
 S^{Y}_{()},
 S^{Z}_{()},
 C^{XY}_{()},
 C^{YZ}_{()},
 C^{ZX}_{()},
 A^{XY}_{()},
 A^{YZ}_{()},
 A^{ZX}_{()},
 H^{X0}_{Gxpi2},
 H^{Y0}_{Gxpi2},
 H^{Z0}_{Gxpi2},
 S^{X0}_{Gxpi2},
 S^{Y0}_{Gxpi2},
 S^{Z0}_{Gxpi2},
 C^{XY0}_{Gxpi2},
 C^{YZ0}_{Gxpi2},
 C^{ZX0}_{Gxpi2},
 A^{XY0}_{Gxpi2},
 A^{YZ0}_{Gxpi2},
 A^{ZX0}_{Gxpi2},
 H^{X0}_{Gypi2},
 H^{Y0}_{Gypi2},
 H^{Z0}_{Gypi2},
 S^{X0}_{Gypi2},
 S^{Y0}_{Gypi2},
 S^{Z0}_{Gypi2},
 C^{XY0}_{Gypi2},
 C^{YZ0}_{Gypi2},
 C^{ZX0}_{Gypi2},
 A^{XY0}_{Gypi2},
 A^{YZ0}_{Gypi2},
 A^{ZX0}_{Gypi2},
 H^{X0}_{Gzpi2},
 H^{Y0}_{Gzpi2},
 H^{Z0}_{Gzpi2},
 S^{X0}_{Gzpi2},
 S^{Y0}_{Gzpi2},
 S^{Z0}_{Gzpi2},
 C^{XY0}_{Gzpi2},
 C^{YZ0}_{Gzpi2},
 C^{ZX0}_{Gzpi2},
 A^{XY0}_{Gzpi2},
 A^{YZ0}_{Gzpi2},
 A^{ZX0}_{Gzpi2},
 H^{XM}_{Mdefault},
 H^{YM}_{Mdefault},
 H^{ZM}_{Mdefault},
 S^{XM}_{Mdefault},
 S^{YM}_{Mdefault},
 S^{ZM}_{Mdefault},
 C^{XYM}_{Mdefault},
 C^{YZM}_{Mdefault},
 C^{ZXM}_{Mdefault},
 A^{XYM}_{Mdefault},
 A^{

In [410]:
class CircuitAnalyzer:
    """Performs operations on a given pygsti Circuit object,
    which is part of a given GateSet."""
    def __init__(self, circuit, gateset):
        self.gateset = gateset
        self.circuit = circuit
        # Extracts the gates in the circuit as a list of labels
        # This may only work for sngle-qubit gates as written?
        self.gates = list(circuit.layertup)
        # Finds their PTMs (pauli transfer matrices)
        self.gate_matrices = [gateset.target_op(gate) for gate in self.gates]
        # Extracts the error matrices corresponding to each of these gates
        self.errors = [gateset.error_generator_of(gate) for gate in self.gates]
        
    def left_errors(self):
        """Moves all gate errors to the left of the circuit,
        and returns the resulting list of conjugated error generators.
        
        NOTE: In our Circuit object, the gates are ordered
        left-to-right from first-applied to last-applied.
        However, the error terms in the result of this function, are ordered
        left-to-right as they would be in matrix multiplication."""
        
        left_errors = []
        # Process the errors from the end of the list to its beginning
        # Because that's the order they'll appear in the resulting matrix mult
        n = len(self.errors)
        for i in range(n-1, -1, -1):
            error = self.errors[i]
            # Conjugate by all gates being applied after i: gates i+1 through n
            for j in range(i+1, n):
                gate_matrix = self.gate_matrices[j].round(10)
                error = gate_matrix * error * np.linalg.inv(gate_matrix)
            left_errors.append(error)
        return left_errors
    
    def linear_F_map(self):
        """Returns the linear approximation of the F map,
        after moving all errors to the left. This approximation assumes they commute.
        The result is a function of the error terms of the gate set.
        """
        # Simply take the sum of all error generators after moving them to the left
        left_errors = self.left_errors()
        return sum(left_errors[1:], left_errors[0])
    
    def order2_F_map(self):
        """Returns the order-2 approximation of the F map,
        after moving all errors to the left, and applying order-2 BCH.
        The result is a function of the error terms of the gate set.
        """
        left_errors = self.left_errors()
        BCH2 = lambda X, Y : X + Y + (X@Y - Y@X) / 2
        # After extracting the left-side errors,
        # reduce the list to one matrix by applying BCH to each pair
        result = left_errors[0]
        for F in left_errors[1:]:
            result = BCH2(result, F)
        return result

In [411]:
std.gates

[(), ('Gxpi2', 0), ('Gypi2', 0), ('Gzpi2', 0)]

In [412]:
gs = GateSet(std)

In [413]:
c.layertup

(Label(('Gxpi2', 0)), Label(('Gzpi2', 0)))

In [414]:
# Create a simple two gate circuit
c = Circuit([("Gxpi2", 0), ("Gzpi2", 0)])
print(c)

Qubit 0 ---|Gxpi2|-|Gzpi2|---



In [415]:
gs.error_generator_of(c[0])

Matrix([
[                   0,                                        0,                                        0,                                        0],
[-4.0*A^{YZ0}_{Gxpi2}, -2.0*S^{Y0}_{Gxpi2} - 2.0*S^{Z0}_{Gxpi2}, 2.0*C^{XY0}_{Gxpi2} - 2.0*H^{Z0}_{Gxpi2}, 2.0*C^{ZX0}_{Gxpi2} + 2.0*H^{Y0}_{Gxpi2}],
[-4.0*A^{ZX0}_{Gxpi2}, 2.0*C^{XY0}_{Gxpi2} + 2.0*H^{Z0}_{Gxpi2}, -2.0*S^{X0}_{Gxpi2} - 2.0*S^{Z0}_{Gxpi2}, 2.0*C^{YZ0}_{Gxpi2} - 2.0*H^{X0}_{Gxpi2}],
[-4.0*A^{XY0}_{Gxpi2}, 2.0*C^{ZX0}_{Gxpi2} - 2.0*H^{Y0}_{Gxpi2}, 2.0*C^{YZ0}_{Gxpi2} + 2.0*H^{X0}_{Gxpi2}, -2.0*S^{X0}_{Gxpi2} - 2.0*S^{Y0}_{Gxpi2}]])

In [426]:
# Initialize our analyzer object. Verify that the error maps for both X(pi/2) gates are identical
a = CircuitAnalyzer(c, gs)
a.errors[1]

Matrix([
[                   0,                                        0,                                        0,                                        0],
[-4.0*A^{YZ0}_{Gzpi2}, -2.0*S^{Y0}_{Gzpi2} - 2.0*S^{Z0}_{Gzpi2}, 2.0*C^{XY0}_{Gzpi2} - 2.0*H^{Z0}_{Gzpi2}, 2.0*C^{ZX0}_{Gzpi2} + 2.0*H^{Y0}_{Gzpi2}],
[-4.0*A^{ZX0}_{Gzpi2}, 2.0*C^{XY0}_{Gzpi2} + 2.0*H^{Z0}_{Gzpi2}, -2.0*S^{X0}_{Gzpi2} - 2.0*S^{Z0}_{Gzpi2}, 2.0*C^{YZ0}_{Gzpi2} - 2.0*H^{X0}_{Gzpi2}],
[-4.0*A^{XY0}_{Gzpi2}, 2.0*C^{ZX0}_{Gzpi2} - 2.0*H^{Y0}_{Gzpi2}, 2.0*C^{YZ0}_{Gzpi2} + 2.0*H^{X0}_{Gzpi2}, -2.0*S^{X0}_{Gzpi2} - 2.0*S^{Y0}_{Gzpi2}]])

In [442]:
a.linear_F_map()

Matrix([
[                                         0,                                                                                    0,                                                                                    0,                                                                                    0],
[-4.0*A^{YZ0}_{Gzpi2} + 4.0*A^{ZX0}_{Gxpi2},   -2.0*S^{X0}_{Gxpi2} - 2.0*S^{Y0}_{Gzpi2} - 2.0*S^{Z0}_{Gxpi2} - 2.0*S^{Z0}_{Gzpi2}, -2.0*C^{XY0}_{Gxpi2} + 2.0*C^{XY0}_{Gzpi2} - 2.0*H^{Z0}_{Gxpi2} - 2.0*H^{Z0}_{Gzpi2}, -2.0*C^{YZ0}_{Gxpi2} + 2.0*C^{ZX0}_{Gzpi2} + 2.0*H^{X0}_{Gxpi2} + 2.0*H^{Y0}_{Gzpi2}],
[-4.0*A^{YZ0}_{Gxpi2} - 4.0*A^{ZX0}_{Gzpi2}, -2.0*C^{XY0}_{Gxpi2} + 2.0*C^{XY0}_{Gzpi2} + 2.0*H^{Z0}_{Gxpi2} + 2.0*H^{Z0}_{Gzpi2},   -2.0*S^{X0}_{Gzpi2} - 2.0*S^{Y0}_{Gxpi2} - 2.0*S^{Z0}_{Gxpi2} - 2.0*S^{Z0}_{Gzpi2},  2.0*C^{YZ0}_{Gzpi2} + 2.0*C^{ZX0}_{Gxpi2} - 2.0*H^{X0}_{Gzpi2} + 2.0*H^{Y0}_{Gxpi2}],
[-4.0*A^{XY0}_{Gxpi2} - 4.0*A^{XY0}_{Gzpi2}, -2.0*C^{YZ0}_{Gxpi2} + 2.0*C^{ZX0}

In [443]:
print(a.linear_F_map())

Matrix([[0, 0, 0, 0], [-4.0*A^{YZ0}_{Gzpi2} + 4.0*A^{ZX0}_{Gxpi2}, -2.0*S^{X0}_{Gxpi2} - 2.0*S^{Y0}_{Gzpi2} - 2.0*S^{Z0}_{Gxpi2} - 2.0*S^{Z0}_{Gzpi2}, -2.0*C^{XY0}_{Gxpi2} + 2.0*C^{XY0}_{Gzpi2} - 2.0*H^{Z0}_{Gxpi2} - 2.0*H^{Z0}_{Gzpi2}, -2.0*C^{YZ0}_{Gxpi2} + 2.0*C^{ZX0}_{Gzpi2} + 2.0*H^{X0}_{Gxpi2} + 2.0*H^{Y0}_{Gzpi2}], [-4.0*A^{YZ0}_{Gxpi2} - 4.0*A^{ZX0}_{Gzpi2}, -2.0*C^{XY0}_{Gxpi2} + 2.0*C^{XY0}_{Gzpi2} + 2.0*H^{Z0}_{Gxpi2} + 2.0*H^{Z0}_{Gzpi2}, -2.0*S^{X0}_{Gzpi2} - 2.0*S^{Y0}_{Gxpi2} - 2.0*S^{Z0}_{Gxpi2} - 2.0*S^{Z0}_{Gzpi2}, 2.0*C^{YZ0}_{Gzpi2} + 2.0*C^{ZX0}_{Gxpi2} - 2.0*H^{X0}_{Gzpi2} + 2.0*H^{Y0}_{Gxpi2}], [-4.0*A^{XY0}_{Gxpi2} - 4.0*A^{XY0}_{Gzpi2}, -2.0*C^{YZ0}_{Gxpi2} + 2.0*C^{ZX0}_{Gzpi2} - 2.0*H^{X0}_{Gxpi2} - 2.0*H^{Y0}_{Gzpi2}, 2.0*C^{YZ0}_{Gzpi2} + 2.0*C^{ZX0}_{Gxpi2} + 2.0*H^{X0}_{Gzpi2} - 2.0*H^{Y0}_{Gxpi2}, -2.0*S^{X0}_{Gxpi2} - 2.0*S^{X0}_{Gzpi2} - 2.0*S^{Y0}_{Gxpi2} - 2.0*S^{Y0}_{Gzpi2}]])
